# 350_Join_FIFA_Fantasy1718 
## Purpose 
In this notebook we read both our Fantasty and FIFA cleaned datasets and join them and performing any necessary cleaning.  
## Datasets 
* _Input_: FIFA18.csv,Fantasy1213.csv
* _Output_: Joined1213.csv

In [1]:
import os.path
import numpy as np
import pandas as pd

### Reading in our cleaned FIFA and Fantasy datasets.

In [2]:
Fifa = pd.read_csv("../../data/prep/FIFA18.csv")
Fantasy1213 = pd.read_csv("../../data/prep/Fantasy1213.csv")

### Merging the two datasets on Surname and First Initial
* Use pd.merge to join the tables  .
* Drop irrelevant column 'Unnamed: 0_x'.

In [3]:
Join1213 = pd.merge(Fifa, Fantasy1213, on=['Surname','FirstInitial'])
Join1213.drop(Join1213.columns[[0]], axis=1, inplace=True)

In [4]:
Join1213.head(5)

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
0,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,2228,32,Spanish Primera Division,1985-02-05,185.0,...,4,5,0,1,0,0,1,34,7.441176,Cristiano
1,158023,L. Messi,Lionel Messi,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2158,30,Spanish Primera Division,1987-06-24,170.0,...,4,0,0,0,0,0,0,32,10.156250,Lionel
2,176580,L. Suarez,Luis Suarez,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2291,30,Spanish Primera Division,1987-01-24,182.0,...,0,2,0,0,0,0,0,33,5.636364,Luis
3,225591,L. Suarez,Leonardo Suarez,Villarreal CF,https://cdn.sofifa.org/18/teams/483.png,1673,21,Spanish Primera Division,1996-03-30,167.0,...,0,2,0,0,0,0,0,33,5.636364,Luis
4,222378,L. Suarez,Lucas Suarez,Defensa y Justicia,https://cdn.sofifa.org/18/teams/111710.png,1521,22,Argentinian Superliga,1995-03-17,181.0,...,0,2,0,0,0,0,0,33,5.636364,Luis


### Creating a while loop to remove players
* Removes all players thats Firstname is not contained in 'name'
* Removes all players thats Firstname is not contained in 'full_name'.

In [5]:
i = 0
array = []

while i < len(Join1213):
    if Join1213['Firstname'][i] in Join1213['name'][i] or Join1213['Firstname'][i] in Join1213['full_name'][i]:
        array.append(True)
        i+=1
    else:
        array.append(False)
        i+=1

### Making the array 'array' a numpy array 
* Making the numpy array into a dataframe.

In [6]:
array = np.array(array)
Join1213 = Join1213[array]

### Creating a dataframe ERROR with all players that have problems
* Contains players in Join that are duplicated using 'Players' or 'full_name'.

In [7]:
Error1213 = Join1213[(Join1213['Players'].isin(Join1213['Players'][Join1213['Players'].duplicated()]) | Join1213['full_name'].isin(Join1213['full_name'][Join1213['full_name'].duplicated()]))]
Error1213.head(5)

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
29,167948,H. Lloris,Hugo Lloris,Tottenham Hotspur,https://cdn.sofifa.org/18/teams/18.png,1318,30,English Premier League,1986-12-26,188.0,...,0,0,0,0,29,11,0,27,2.925926,Hugo
30,167948,H. Lloris,Hugo Lloris,Tottenham Hotspur,https://cdn.sofifa.org/18/teams/18.png,1318,30,English Premier League,1986-12-26,188.0,...,0,0,0,0,2,0,0,2,2.000000,Hugo
40,176676,Marcelo,Marcelo Vieira da Silva,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,2214,29,Spanish Primera Division,1988-05-12,174.0,...,1,4,0,0,15,4,0,14,2.357143,Marcelo
41,180334,Marcelo,Marcelo Antonio Guedes Filho,Olympique Lyonnais,https://cdn.sofifa.org/18/teams/66.png,1789,30,French Ligue 1,1987-05-20,191.0,...,1,4,0,0,15,4,0,14,2.357143,Marcelo
42,205340,Marcelo,Marcelo dos Santos Ferreira,Rio Ave FC,https://cdn.sofifa.org/18/teams/744.png,1625,27,Portuguese Primeira Liga,1989-07-27,182.0,...,1,4,0,0,15,4,0,14,2.357143,Marcelo


### Creating a dataframe TopLeagues with all players inside the Top 5 leagues
* Takes all players in ERROR insode the Top 5 leagues using str.contains.

In [8]:
leagues = ['German Bundesliga','Spanish Primera Division','English Premier League','French Ligue 1','Italian Serie A']
TopLeagues1213 = Error1213[(Error1213.league.str.contains('|'.join(leagues)) == True) | Error1213['league'].isnull()]

### Creating a dataframe DELETE with all players outside the Top 5 leagues
* Takes all players in ERROR outside the Top 5 leagues using str.contains.

In [9]:
Delete1213 = Error1213[Error1213.league.str.contains('|'.join(leagues)) == False]

### Creating a dataframe FIX
* This dataframe takes all players in the Top 5 that have been duplicated .

In [10]:
Fix1213 = TopLeagues1213[(TopLeagues1213['Players'].isin(TopLeagues1213['Players'][TopLeagues1213['Players'].duplicated()]) | TopLeagues1213['full_name'].isin(TopLeagues1213['full_name'][TopLeagues1213['full_name'].duplicated()]))]
Fix1213.head(5)

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
29,167948,H. Lloris,Hugo Lloris,Tottenham Hotspur,https://cdn.sofifa.org/18/teams/18.png,1318,30,English Premier League,1986-12-26,188.0,...,0,0,0,0,29,11,0,27,2.925926,Hugo
30,167948,H. Lloris,Hugo Lloris,Tottenham Hotspur,https://cdn.sofifa.org/18/teams/18.png,1318,30,English Premier League,1986-12-26,188.0,...,0,0,0,0,2,0,0,2,2.000000,Hugo
40,176676,Marcelo,Marcelo Vieira da Silva,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,2214,29,Spanish Primera Division,1988-05-12,174.0,...,1,4,0,0,15,4,0,14,2.357143,Marcelo
41,180334,Marcelo,Marcelo Antonio Guedes Filho,Olympique Lyonnais,https://cdn.sofifa.org/18/teams/66.png,1789,30,French Ligue 1,1987-05-20,191.0,...,1,4,0,0,15,4,0,14,2.357143,Marcelo
167,239335,Oscar,Oscar Rodriguez Arnaiz,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,1667,19,Spanish Primera Division,1998-06-28,174.0,...,10,11,0,0,0,0,0,34,2.764706,Oscar


### Removing players from JOIN
* Removing players whos full name is in Delete and have the same birth date.
* Removing players whos full name is in Fix and have the same birth date.

In [11]:
Join1213 = Join1213[(~(Join1213.full_name.isin(Delete1213.full_name) & Join1213.birth_date.isin(Delete1213.birth_date)))]
Join1213 = Join1213[(~(Join1213.full_name.isin(Fix1213.full_name) & Join1213.birth_date.isin(Fix1213.birth_date)))]

### Removing players from FIX
* Removing players thats 'full_name' is not equal to 'Players'.
* Dropping all duplicate players in Fixed.

In [12]:
Fixed1213 = Fix1213[Fix1213['full_name'] == Fix1213['Players']]
Fixed1213 = Fixed1213[(Fixed1213['Players'].isin(Fixed1213['Players'][Fixed1213['Players'].duplicated()]) | Fixed1213['full_name'].isin(Fixed1213['full_name'][Fixed1213['full_name'].duplicated()]))]
Fixed1213 = Fixed1213[~Fixed1213.duplicated(['Players', 'Clubs', 'Points'])]
Fixed1213.head(5)

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
29,167948,H. Lloris,Hugo Lloris,Tottenham Hotspur,https://cdn.sofifa.org/18/teams/18.png,1318,30,English Premier League,1986-12-26,188.0,...,0,0,0,0,29,11,0,27,2.925926,Hugo
30,167948,H. Lloris,Hugo Lloris,Tottenham Hotspur,https://cdn.sofifa.org/18/teams/18.png,1318,30,English Premier League,1986-12-26,188.0,...,0,0,0,0,2,0,0,2,2.000000,Hugo
223,204639,S. Savic,Stefan Savic,Atletico Madrid,https://cdn.sofifa.org/18/teams/240.png,1692,26,Spanish Primera Division,1991-01-08,187.0,...,0,0,0,0,0,0,0,0,0.000000,Stefan
224,204639,S. Savic,Stefan Savic,Atletico Madrid,https://cdn.sofifa.org/18/teams/240.png,1692,26,Spanish Primera Division,1991-01-08,187.0,...,0,3,0,0,33,16,0,26,2.884615,Stefan
245,186627,M. Balotelli,Mario Balotelli,OGC Nice,https://cdn.sofifa.org/18/teams/72.png,1889,26,French Ligue 1,1990-08-12,189.0,...,7,6,0,0,0,0,0,14,1.928571,Mario


### Adding players who have transfered teams
* Get the sum of all relevant columns to get one full player

In [13]:
d = {
 'ID' : 'first',
 'club_logo':'first',
 'special':'first',
 'age':'first',
 'league':'first',
 'birth_date':'first',
 'height_cm':'first',
 'weight_kg':'first',
 'body_type':'first',
 'real_face':'first',
 'flag':'first',
 'nationality':'first',
 'photo':'first',
 'eur_value':'first',
 'eur_wage':'first',
 'eur_release_clause':'first',
 'overall':'first',
 'potential':'first',
 'pac':'first',
 'sho':'first',
 'pas':'first',
 'dri':'first',
 'def':'first',
 'phy':'first',
 'international_reputation':'first',
 'skill_moves':'first',
 'weak_foot':'first',
 'work_rate_att':'first',
 'work_rate_def':'first',
 'preferred_foot':'first',
 'crossing':'first',
 'finishing':'first',
 'heading_accuracy':'first',
 'short_passing':'first',
 'volleys':'first',
 'dribbling':'first',
 'curve':'first',
 'free_kick_accuracy':'first',
 'long_passing':'first',
 'ball_control':'first',
 'acceleration':'first',
 'sprint_speed':'first',
 'agility':'first',
 'reactions':'first',
 'balance':'first',
 'shot_power':'first',
 'jumping':'first',
 'stamina':'first',
 'strength':'first',
 'long_shots':'first',
 'aggression':'first',
 'interceptions':'first',
 'positioning':'first',
 'vision':'first',
 'penalties':'first',
 'composure':'first',
 'marking':'first',
 'standing_tackle':'first',
 'sliding_tackle':'first',
 'gk_diving':'first',
 'gk_handling':'first',
 'gk_kicking':'first',
 'gk_positioning':'first',
 'gk_reflexes':'first',
 'rs':'first',
 'rw':'first',
 'rf':'first',
 'ram':'first',
 'rcm':'first',
 'rm':'first',
 'rdm':'first',
 'rcb':'first',
 'rb':'first',
 'rwb':'first',
 'st':'first',
 'lw':'first',
 'cf':'first',
 'cam':'first',
 'cm':'first',
 'lm':'first',
 'cdm':'first',
 'cb':'first',
 'lb':'first',
 'lwb':'first',
 'ls':'first',
 'lf':'first',
 'lam':'first',
 'lcm':'first',
 'ldm':'first',
 'lcb':'first',
 'gk':'first',
 '1_on_1_rush_trait':'first',
 'acrobatic_clearance_trait':'first',
 'argues_with_officials_trait':'first',
 'avoids_using_weaker_foot_trait':'first',
 'backs_into_player_trait':'first',
 'bicycle_kicks_trait':'first',
 'cautious_with_crosses_trait':'first',
 'chip_shot_trait':'first',
 'chipped_penalty_trait':'first',
 'comes_for_crosses_trait':'first',
 'corner_specialist_trait':'first',
 'diver_trait':'first',
 'dives_into_tackles_trait':'first',
 'diving_header_trait':'first',
 'driven_pass_trait':'first',
 'early_crosser_trait':'first',
 "fan's_favourite_trait":'first',
 'fancy_flicks_trait':'first',
 'finesse_shot_trait':'first',
 'flair_trait':'first',
 'flair_passes_trait':'first',
 'gk_flat_kick_trait':'first',
 'gk_long_throw_trait':'first',
 'gk_up_for_corners_trait':'first',
 'giant_throw_in_trait':'first',
 'inflexible_trait':'first',
 'injury_free_trait':'first',
 'injury_prone_trait':'first',
 'leadership_trait':'first',
 'long_passer_trait':'first',
 'long_shot_taker_trait':'first',
 'long_throw_in_trait':'first',
 'one_club_player_trait':'first',
 'outside_foot_shot_trait':'first',
 'playmaker_trait':'first',
 'power_free_kick_trait':'first',
 'power_header_trait':'first',
 'puncher_trait':'first',
 'rushes_out_of_goal_trait':'first',
 'saves_with_feet_trait':'first',
 'second_wind_trait':'first',
 'selfish_trait':'first',
 'skilled_dribbling_trait':'first',
 'stutter_penalty_trait':'first',
 'swerve_pass_trait':'first',
 'takes_finesse_free_kicks_trait':'first',
 'target_forward_trait':'first',
 'team_player_trait':'first',
 'technical_dribbler_trait':'first',
 'tries_to_beat_defensive_line_trait':'first',
 'poacher_speciality':'first',
 'speedster_speciality':'first',
 'aerial_threat_speciality':'first',
 'dribbler_speciality':'first',
 'playmaker_speciality':'first',
 'engine_speciality':'first',
 'distance_shooter_speciality':'first',
 'crosser_speciality':'first',
 'free_kick_specialist_speciality':'first',
 'tackling_speciality':'first',
 'tactician_speciality':'first',
 'acrobat_speciality':'first',
 'strength_speciality':'first',
 'clinical_finisher_speciality':'first',
 'prefers_rs':'first',
 'prefers_rw':'first',
 'prefers_rf':'first',
 'prefers_ram':'first',
 'prefers_rcm':'first',
 'prefers_rm':'first',
 'prefers_rdm':'first',
 'prefers_rcb':'first',
 'prefers_rb':'first',
 'prefers_rwb':'first',
 'prefers_st':'first',
 'prefers_lw':'first',
 'prefers_cf':'first',
 'prefers_cam':'first',
 'prefers_cm':'first',
 'prefers_lm':'first',
 'prefers_cdm':'first',
 'prefers_cb':'first',
 'prefers_lb':'first',
 'prefers_lwb':'first',
 'prefers_ls':'first',
 'prefers_lf':'first',
 'prefers_lam':'first',
 'prefers_lcm':'first',
 'prefers_ldm':'first',
 'prefers_lcb':'first',
 'prefers_gk':'first',
 'Players':'first',
 'Position':'first',
 'Clubs':'first',
 'Points':'sum',  
 'Start':'sum', 
 'Sub':'sum', 
 'club':'last',
 'Position':'first',
 'Start':'sum',
 'Goals':'sum',
 'A':'sum',
 'CS':'sum',
 'CS part':'sum',
 'Yellow':'sum',
 'Red':'sum',
 'Sub':'sum',
 'CO':'sum',
 'Off':'sum',
 'Pen SV':'sum',
 'Pen M':'sum',
 'Goals conceded':'sum',
 'Conceded 1+':'sum',
 'OG':'sum',
 'Firstname':'first',
 'Apps':'sum',
 'Form': 'mean'
}

#### Groupby function applying 'd' to fix joining the players.

In [14]:
Fixed1213 = (Fixed1213.groupby(['name','full_name'], sort=False, as_index=False).agg(d).reindex(columns=Fix1213.columns))
Fixed1213.head(5)

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
0,167948,H. Lloris,Hugo Lloris,Tottenham Hotspur,https://cdn.sofifa.org/18/teams/18.png,1318,30,English Premier League,1986-12-26,188.0,...,0,0,0,0,31,11,0,29,2.462963,Hugo
1,204639,S. Savic,Stefan Savic,Atletico Madrid,https://cdn.sofifa.org/18/teams/240.png,1692,26,Spanish Primera Division,1991-01-08,187.0,...,0,3,0,0,33,16,0,26,1.442308,Stefan
2,186627,M. Balotelli,Mario Balotelli,OGC Nice,https://cdn.sofifa.org/18/teams/72.png,1889,26,French Ligue 1,1990-08-12,189.0,...,8,6,0,0,0,0,0,27,4.002747,Mario
3,167431,G. Castro,Gonzalo Castro,Borussia Dortmund,https://cdn.sofifa.org/18/teams/22.png,2156,30,German Bundesliga,1987-06-11,172.0,...,2,14,0,0,38,16,0,31,3.677419,Gonzalo
4,205362,M. Nastasic,Matija Nastasic,FC Schalke 04,https://cdn.sofifa.org/18/teams/34.png,1730,24,German Bundesliga,1993-03-28,188.0,...,0,2,0,0,19,8,0,22,2.833333,Matija


#### Quick look at what FIXED with most relevant columns

In [15]:
Fixed1213[['full_name','club','Players','Clubs','Points']]

,full_name,club,Players,Clubs,Points
0,Hugo Lloris,Tottenham Hotspur,Hugo Lloris,SPURS,83
1,Stefan Savic,Atletico Madrid,Stefan Savic,MCFC,75
2,Mario Balotelli,OGC Nice,Mario Balotelli,MCFC,106
3,Gonzalo Castro,Borussia Dortmund,Gonzalo Castro,BL1,114
4,Matija Nastasic,FC Schalke 04,Matija Nastasic,MCFC,79
5,Federico Fernandez,Swansea City,Federico Fernandez,NAPO,49
6,Nuri Sahin,Borussia Dortmund,Nuri Sahin,LPOOL,70
7,David Abraham,Eintracht Frankfurt,David Abraham,NaN,45
8,Moussa Sissoko,Tottenham Hotspur,Moussa Sissoko,NEWU,84
9,Loic Remy,UD Las Palmas,Loic Remy,QPR,77


### Joining Join and Fixed
* Using pd.concat to join the two dataframes together.
* Drop irrelevant column.

In [16]:
Joined1213 = pd.concat([Join1213,Fixed1213])
Joined1213 = Joined1213.reset_index()
Joined1213.drop(Joined1213.columns[[0]], axis=1, inplace=True)
Joined1213.head(5)

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
0,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,2228,32,Spanish Primera Division,1985-02-05,185.0,...,4,5,0,1,0,0,1,34,7.441176,Cristiano
1,158023,L. Messi,Lionel Messi,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2158,30,Spanish Primera Division,1987-06-24,170.0,...,4,0,0,0,0,0,0,32,10.156250,Lionel
2,176580,L. Suarez,Luis Suarez,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2291,30,Spanish Primera Division,1987-01-24,182.0,...,0,2,0,0,0,0,0,33,5.636364,Luis
3,167495,M. Neuer,Manuel Neuer,FC Bayern Munich,https://cdn.sofifa.org/18/teams/21.png,1493,31,German Bundesliga,1986-03-27,193.0,...,0,0,1,0,18,5,0,31,4.258065,Manuel
4,188545,R. Lewandowski,Robert Lewandowski,FC Bayern Munich,https://cdn.sofifa.org/18/teams/21.png,2146,28,German Bundesliga,1988-08-21,185.0,...,2,1,0,1,0,0,0,31,6.064516,Robert


#### Saving to csv file in data/prep

In [17]:
Joined1213.to_csv('../../data/prep/Joined1213.csv')